In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import tf_keras 
import tensorflow_hub as hub

model4_path = '/kaggle/input/model4-final/model4.h5'
model4 = tf_keras.models.load_model(model4_path, custom_objects={'KerasLayer': hub.KerasLayer})

In [12]:
unique_labels = ['freshapples','freshbanana','freshbittergroud','freshcapsicum','freshcucumber','freshokra', 
               'freshoranges','freshpotato','freshtomato','rottenapples','rottenbanana','rottenbittergroud',
               'rottencapsicum','rottencucumber','rottenokra','rottenoranges','rottenpotato','rottentomato'];


In [6]:
import tensorflow as tf
import tensorflow_hub as hub

Img_size = 224 ;

def process_image(image_path,image_size = Img_size):

    image = tf.io.read_file(image_path)

    image = tf.image.decode_jpeg(image,channels = 3)

    image = tf.image.convert_image_dtype(image,tf.float32) 

    image = tf.image.resize(image,size = [image_size,image_size])

    return image ;

def get_image_and_label(image_path,label):
    image = process_image(image_path)
    return image,label

  
Batch_size = 32 ;
 
def Create_data_batches(x,y=None,batch_size = Batch_size,val_data = False , test_data = False):
    if test_data:
        print("Creating Test Data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) ;
        data = data.map(process_image) ;
        data_batch = data.batch(batch_size);
        return data_batch
    elif  val_data:
        print("Creating Validation Data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x),tf.constant(y))) ;
        data = data.map(get_image_and_label) ;
        data_batch = data.batch(batch_size);
        return data_batch ;
    else :
         print("Creating Training Data batches...")
         data = tf.data.Dataset.from_tensor_slices((tf.constant(x),tf.constant(y))) ;
         data = data.shuffle(buffer_size = len(x)) ;
         data = data.map(get_image_and_label);
         data_batch = data.batch(batch_size);
         return data_batch

In [7]:
import numpy as np
filenames = [] ;
file_path = '/kaggle/input/1-image/Screenshot 2024-10-20 120247.png' ;
filenames.append(file_path);
filenames = np.array(filenames);

In [8]:
Test_data = Create_data_batches(filenames,test_data=True) ;

Creating Test Data batches...


In [9]:
pred = model4.predict(Test_data);

1/1 [==============================] - 10s 10s/step


In [11]:
def get_predicted_label(prediction_probs):
    return unique_labels[np.argmax(prediction_probs)]

In [14]:
pred_label = get_predicted_label(pred);
print(pred_label);

freshoranges
